# Modelizar y crear función de recomendación

### Importar librerias

In [1]:
import pandas as pd
import numpy as np


### Cargar Dataset

In [2]:
#Importo dataset
df = pd.read_parquet('datasets_ml.parquet')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32184 entries, 0 to 32183
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            32184 non-null  int64  
 1   title         32184 non-null  object 
 2   release_year  32184 non-null  int32  
 3   genres        30543 non-null  object 
 4   tagline       17914 non-null  object 
 5   overview      32184 non-null  object 
 6   vote_average  32184 non-null  float64
 7   actores       30295 non-null  object 
 8   director      31500 non-null  object 
dtypes: float64(1), int32(1), int64(1), object(6)
memory usage: 2.1+ MB


#### Eliminar valores nulos

In [3]:
df = df[df['genres'].notna()]
df = df[df['actores'].notna()]

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29216 entries, 0 to 32183
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            29216 non-null  int64  
 1   title         29216 non-null  object 
 2   release_year  29216 non-null  int32  
 3   genres        29216 non-null  object 
 4   tagline       17378 non-null  object 
 5   overview      29216 non-null  object 
 6   vote_average  29216 non-null  float64
 7   actores       29216 non-null  object 
 8   director      28946 non-null  object 
dtypes: float64(1), int32(1), int64(1), object(6)
memory usage: 2.1+ MB


In [5]:
df.columns

Index(['id', 'title', 'release_year', 'genres', 'tagline', 'overview',
       'vote_average', 'actores', 'director'],
      dtype='object')

#### Seleccionar peliculas por año

Como en el EDA, hay un incremento en las películas registradas en el dataset a partir de 1990. Intenté reducir en ese tamaño el dataset pero generó una matriz muy grande. Asi que reduje el tamaño tomando peliculas más modernas

In [6]:
mask = df['release_year'] > 1999
df =  df[mask]

#### Unir características útiles

In [7]:
#Concateno informacion a usar
df['features'] = df['genres'].str.cat([df['overview'], df['director']], sep=' ')

In [8]:
df['features'].info()

<class 'pandas.core.series.Series'>
Index: 14844 entries, 15536 to 32183
Series name: features
Non-Null Count  Dtype 
--------------  ----- 
14632 non-null  object
dtypes: object(1)
memory usage: 231.9+ KB


In [9]:
df

,id,title,release_year,genres,tagline,overview,vote_average,actores,director,features
15536,38987,possible worlds,2000,mysteri scienc fiction drama,None,man live sever parallel live differ world diff...,5.8,tom mccamu tilda swinton shaun mccann gabriel ...,robert lepag,mysteri scienc fiction drama man live sever pa...
15537,4369,just a question of love,2000,drama romanc,None,gay cousin die hepat young laurent live best f...,7.5,cyril thouvenin phan rin tilli carolin veyt ev...,christian faur,drama romanc gay cousin die hepat young lauren...
15538,59317,scrapbook,2000,drama horror,None,young woman name clara captur serial killer na...,5.3,emili haack tommi biondo,eric stanz,drama horror young woman name clara captur ser...
15539,27859,rated x,2000,drama,true stori mitchel brother america origin porn...,base true stori jim arti mitchel two brother e...,5.2,charli sheen emilio estevez traci hutson nicol...,emilio estevez,drama base true stori jim arti mitchel two bro...
15541,46119,crime + punishment in suburbia,2000,drama,one stay innoc forev,contemporari fabl loos base fyodor dostoyevski...,6.1,monica keena ellen barkin michael ironsid vinc...,robert schmidt,drama contemporari fabl loos base fyodor dosto...
...,...,...,...,...,...,...,...,...,...,...
32179,412059,mobile homes,2018,drama,None,forgotten town along american border young mot...,0.0,imogen poot callum turner callum keith renni f...,vladimir fontenay,drama forgotten town along american border you...
32180,332283,mary shelley,2018,drama romanc,None,love affair poet perci shelley year old mari w...,0.0,ell fan dougla booth bel powley maisi william ...,haifaa mansour,drama romanc love affair poet perci shelley ye...
32181,38700,bad boys for life,2018,thriller action crime,None,continu adventur miami detect mike lowrey marc...,0.0,smith martin lawrenc derrick gilbert,joe carnahan,thriller action crime continu adventur miami d...
32182,299782,the other side of the wind,2018,comedi drama,None,orson well unfinish masterpiec restor assembl ...,0.0,john huston robert random peter bogdanovich su...,orson well,comedi drama orson well unfinish masterpiec re...


#### Seleccionamos los campos a utilizar para entrenar

In [10]:
df = df[['id','title','features']]

ELiminar datos nulos

In [11]:
df.dropna(inplace=True)

/tmp/ipykernel_1759112/1379821321.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14632 entries, 15536 to 32183
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        14632 non-null  int64 
 1   title     14632 non-null  object
 2   features  14632 non-null  object
dtypes: int64(1), object(2)
memory usage: 457.2+ KB


### Entrenamiento del modelo

Utilizo un modelo TF-IDF para vectorizar los textos de las características seleccionadas. Creando una matriz de vectores para utilizar en el cálculo de similitud del coseno

In [13]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Crea un vectorizador TF-IDF
vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))

# Ajusta el vectorizador a la columna 'overview' y transforma los datos
tfidf_matrix = vectorizer.fit_transform(df['features'])

# Calcula la similitud coseno entre los vectores
similarity_matrix = cosine_similarity(tfidf_matrix)



In [27]:
# Define una función para obtener las películas similares a una película dada
def get_similar_movies(movie, N=5):

    #Poner en minúsculas
    movie = movie.lower()
    # Buscar la película en la base de datos
    movie_id = df.loc[df['title'] == movie, 'id'].values[0]
    # Obtiene el índice de la película en la matriz de similitud
    idx = np.where(df['id'] == movie_id)[0][0]
    
    # Obtiene las puntuaciones de similitud para la película
    scores = list(enumerate(similarity_matrix[idx]))
    
    # Ordena las puntuaciones en orden descendente
    scores.sort(key=lambda x: x[1], reverse=True)
    
    # Obtiene las películas similares
    similar_movies = [df.iloc[i,1] for i, _ in scores[1:N+1]]
    
    return similar_movies


In [26]:
# Ejemplo de uso:
movie = 'avatar' 
# int(df['id'][df['title'] == movie])
# df.loc[df['title'] == movie, 'id']
df.loc[df['title'] == movie, 'id'].values[0]

np.int64(19995)

In [28]:
# Ejemplo de uso:
movie = 'avatar'  # Reemplaza con el ID de la película para la que deseas obtener recomendaciones
similar_movies = get_similar_movies(movie)

print(similar_movies)

['avatar 2', 'aliens of the deep', 'ghosts of the abyss', 'avatar: creating the world of pandora', 'deepsea challenge 3d']


In [19]:
df.to_parquet('ml.parquet',engine='pyarrow')

# Prueba main

In [ ]:
import pandas as pd
import unicodedata
import numpy as np

ml =  pd.read_parquet('ml.parquet')

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Crea un vectorizador TF-IDF
vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))

# Ajusta el vectorizador a la columna 'overview' y transforma los datos
tfidf_matrix = vectorizer.fit_transform(ml['features'])

# Calcula la similitud coseno entre los vectores
similarity_matrix = cosine_similarity(tfidf_matrix)

#Funcion de recomendacion
def recomendadorAPI(titulo: str):

    #Poner titulo en minúsculas
    titulo = titulo.lower()
    # Obtener el id de la película
    movie_id =  ml.loc[ml['title'] == titulo, 'id'].values[0]
    # Obtiene el índice de la película en la matriz de similitud
    idx = np.where(ml['id'] == movie_id)[0][0]
    
    # Obtiene las puntuaciones de similitud para la película
    scores = list(enumerate(similarity_matrix[idx]))
    
    # Ordena las puntuaciones en orden descendente
    scores.sort(key=lambda x: x[1], reverse=True)
    
    # Obtiene las películas similares
    similar_movies = [ml.iloc[i,1] for i, _ in scores[1:N+1]]
    
    return similar_movies